In [1]:
from __future__ import print_function

In [2]:
import numpy as np
from axidma import AxiDMA


def to_bin(x):
    vv = [ (x>>(32-(i+1)*4))&0xF for i in range(8)]
    return '{0:04b}_{1:04b}|{2:04b}_{3:04b}|{4:04b}_{5:04b}|{6:04b}_{7:04b}'.format( *vv )



dma = AxiDMA('dma')

dma

UIO: dma (uio2) sz:0x10000 @0x40400000

In [3]:
print('\n'.join([to_bin(x) for x in [dma.mm2s.cr, dma.s2mm.cr, dma.mm2s.st,dma.s2mm.st]]))

0000_0000|0000_0001|0000_0000|0000_0010
0000_0000|0000_0001|0000_0000|0000_0010
0000_0000|0000_0000|0000_0000|0000_0001
0000_0000|0000_0000|0000_0000|0000_0001


In [4]:
import uio
reload(uio)
mem_uio = uio.UIO('scratch_mem')

SZ=(1<<19)

data = mem_uio.as_ndarray()

src_data = data[:SZ]
dst_data = data[SZ:SZ*2]

src_data[:] = np.random.randint(0,256,SZ)
dst_data[:] = 0xFF

print("SRC:",src_data[:4])
print("DST:",dst_data[:4])

SRC: [ 19 105   7 192]
DST: [255 255 255 255]


In [5]:
dma.launch(mem_uio.phy_buf(src_data), mem_uio.phy_buf(dst_data), enable_interrupts=True)
if dma.wait():
    print('DMA Completed Succesfully')
else:
    print('DMA FAILED')



DMA Completed Succesfully


In [6]:
if (src_data == dst_data).all():
    print('SUCCESS: Data copied as expected')
else:
    print('FAILED: dst and src do not match up')

print("SRC:",src_data[:4],'...')
print("DST:",dst_data[:4],'...')

SUCCESS: Data copied as expected
SRC: [ 19 105   7 192] ...
DST: [ 19 105   7 192] ...


In [7]:
%%sh

date
whoami
ls -l /dev/uio* /dev/mem
cat /sys/class/uio/uio?/maps/map0/name
cat /sys/class/uio/uio?/name
ls -l /sys/class/uio/uio0/maps/map0

Mon Jan 18 12:13:22 UTC 2016
parallella
crw-r----- 1 root kmem      1, 1 Jan  1  1970 /dev/mem
crw-rw---- 1 root plugdev 247, 0 Jan  1  1970 /dev/uio0
crw-rw---- 1 root plugdev 247, 1 Jan  1  1970 /dev/uio1
crw-rw---- 1 root plugdev 247, 2 Jan  1  1970 /dev/uio2
/scratch_mem@3e000000
/amba_pl/my_mult@70020000
/amba_pl/dma@40400000
scratch_mem
my_mult
dma
total 0
-r--r--r-- 1 root root 4096 Jan 10 12:56 addr
-r--r--r-- 1 root root 4096 Jan 10 12:56 name
-r--r--r-- 1 root root 4096 Jan 10 12:56 offset
-r--r--r-- 1 root root 4096 Jan 10 12:56 size


In [ ]:
#!dtc -I fs -O dts /proc/device-tree/ 